In [3]:
import pandas as pd
import networkx as nx
import numpy as np
import os
from db_con.pcn_oracle_data import PCNDB
from tqdm import tqdm
os.getcwd()

'/home/wzk/Code/PCN_ROUTES'

In [4]:
pcnTestDB = PCNDB('PCN_TEST','784427618', '10.112.199.111:1521/LHR11G')

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "libclntsh.so: cannot open shared object file: No such file or directory". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help

In [21]:
bus_inf = pcnTestDB.get_all_businesses()
cha_inf = pcnTestDB.get_all_channels()

In [22]:
def get_stn_name(stn_id, pcnDB, stnDict):
    del_full_name = lambda x:x[0].split('/')[-1] if x[0] != None else None
    if stn_id not in stnDict.keys():
        stn_dict = pcnDB.get_station_inf(stn_id, tCols=['FULL_NAME'])
        stn_name = del_full_name(stn_dict['FULL_NAME']) if stn_dict['FULL_NAME'] != None else None
        stnDict[stn_id] = stn_name
    else:
        stn_name = stnDict[stn_id]
    
    return stn_name, stnDict


def ch_channel_name(channelInf, pcnDB):
    assert 'A_STATION' in channelInf.keys() and 'Z_STATION' in channelInf.keys()
    stnList_a = channelInf['A_STATION']
    stnList_z = channelInf['Z_STATION']

    stnList_a_name , stnList_z_name = [], []
    resDict = {}

    for idx, (s_a, s_z) in enumerate(zip(stnList_a, stnList_z)):
        s_a_name, resDict = get_stn_name(s_a, pcnDB, resDict)
        s_z_name, resDict = get_stn_name(s_z, pcnDB, resDict)

        stnList_a_name.append(s_a_name)
        stnList_z_name.append(s_z_name)

    channelInf['A_STATION_NAME'] = stnList_a_name
    channelInf['Z_STATION_NAME'] = stnList_z_name

    return channelInf, resDict


def del_channel_inf(channelInf, pcnDB):
    channelInf, stnDict = ch_channel_name(channelInf, pcnDB)
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']

    assert len(stnList_a) == len(stnList_z)
    idx = 0
    while idx < len(stnList_a):
        s_a = stnList_a[idx]
        s_z = stnList_z[idx]
        if s_a is None or s_z is None:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        elif s_a == s_z:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        else:
            idx += 1

    return channelInf, stnDict


def con_business_inf(b_id, pcnDB, stnDict):
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z = get_stn_name(stnID_z, pcnDB, stnDict)
    busInfDict['A_STATION_NAME'] = stnName_a
    busInfDict['Z_STATION_NAME'] = stnName_z

    return busInfDict


def con_business_stn_inf(b_id, pcnDB, stnDict):
    resDitc={}
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a,_ = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z,_ = get_stn_name(stnID_z, pcnDB, stnDict)
    resDitc['A_STATION_NAME'] = stnName_a
    resDitc['Z_STATION_NAME'] = stnName_z

    return resDitc

 
def del_business_inf(businessInf, pcnDB, stnDict=None):
    assert 'BUSINESS_ID' in businessInf.keys() and 'CHANNEL_ID' in businessInf
    busDict = {}
    for idx, busID in enumerate(businessInf['BUSINESS_ID']):
        if busID not in busDict.keys():
            busDict[busID] = [businessInf['CHANNEL_ID'][idx]]
        else:
            busDict[busID].append(businessInf['CHANNEL_ID'][idx])

    businessInfDict ={}
    for idx, b_id in enumerate(list(busDict.keys())):
        busInfDict = con_business_stn_inf(b_id, pcnDB, stnDict)
        businessInfDict[b_id] = busInfDict

    return busDict, businessInfDict


def generate_pcnMap(channelInf):
    pcnMap = nx.Graph()
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']
    channelIdList = channelInf['OBJ_ID']

    edgeDict = {}
    for idx, (c_id, s_a, s_z) in enumerate(zip(channelIdList, stnList_a, stnList_z)):
        edgeStr = '{}-{}'.format(s_a, s_z)
        edgeStrRe = '{}-{}'.format(s_z, s_a)
        if edgeStr in edgeDict.keys():
            edgeDict[edgeStr].append(c_id)
        elif edgeStrRe in edgeDict.keys():
            edgeDict[edgeStrRe].append(c_id)
        else:
            pcnMap.add_edge(s_a, s_z)
            edgeDict[edgeStr] = [c_id]

    for e in pcnMap.edges.keys():
        edgekey = '{}-{}'.format(e[0], e[1])
        edgekeyRe = '{}-{}'.format(e[1], e[0])
        if edgekey in edgeDict.keys():
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekey]
        else:
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekeyRe]


    return pcnMap


def get_circuitous_bus(businessInfDict, pcnMap):
    busDelDict={}
    for idx,b_id in enumerate(businessInfDict.keys()):
        stnName_a = businessInfDict[b_id]['A_STATION_NAME']
        stnName_z = businessInfDict[b_id]['Z_STATION_NAME']

        if stnName_a not in pcnMap.nodes.keys() or stnName_z not in pcnMap.nodes.keys():
            continue
        elif stnName_a is None or stnName_z is None:
            continue
        elif stnName_a == stnName_z:
            continue
        else:
            busDelDict[b_id] = businessInfDict[b_id]

    return busDelDict

cha_inf_s, stnDict = del_channel_inf(cha_inf, pcnTestDB)
bus_inf_s,  bus_inf2_s = del_business_inf(bus_inf, pcnTestDB, stnDict)
pcnMap = generate_pcnMap(cha_inf_s)
busDelDict = get_circuitous_bus(bus_inf2_s, pcnMap)


In [23]:
def get_access_routes(busDelDict, pcnMap, cutoff=3):
    resDict={}
    routeDict = {}
    busIDList = list(busDelDict.keys())
    for idx in tqdm(range(len(busDelDict))):
        b_id = busIDList[idx]
        stn_a, stn_z = busDelDict[b_id]['A_STATION_NAME'], busDelDict[b_id]['Z_STATION_NAME']
        resKey = "{}-{}".format(stn_a, stn_z)
        resKey1 = "{}-{}".format(stn_z, stn_a)
        if resKey in resDict.keys():
            routeDict[b_id] = resDict[resKey]
        elif resKey1 in resDict.keys():
            routeDict[b_id] = resDict[resKey1]
        else:
            nx_allRoutes = nx.all_simple_paths(pcnMap,source=stn_a,target=stn_z, cutoff=cutoff)
            routes = list(nx_allRoutes)
            routeDict[b_id] = routes
            resDict[resKey] = routes

    return routeDict
    
routeDict = get_access_routes(busDelDict, pcnMap)

100%|██████████| 18083/18083 [00:40<00:00, 445.52it/s]


In [69]:
np.array(list(dict(pcnMap.degree).values()))

array([ 7, 26,  4, ...,  1,  1,  1])

In [86]:
print("{:.2f}".format(np.array([d for _, d in pcnMap.degree]).mean()))


7.82


In [29]:
print(" MAP INF ".center(50, "*"))
print('\t* Nodes num:{}'.format(len(pcnMap.nodes)))
print('\t* Edges num:{}'.format(len(pcnMap.edges)))
print(" MAP INF ".center(50,'*'))

******************** MAP INF *********************
	* Nodes num:3048
	* Edges num:11911
******************** MAP INF *********************


In [13]:
c, v = pcnTestDB._exec_sql("SELECT * FROM T_SPC_SITE WHERE OBJ_ID = 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391'")

In [49]:
bus_inf_s['94FAF1DB-56F7-4C1E-B539-A542C1474E37-29055']

['F8165186-C245-4465-8A2A-DB5C9ABE134D-44585']

In [51]:
cha_inf['F8165186-C245-4465-8A2A-DB5C9ABE134D-44585']


KeyError: 'F8165186-C245-4465-8A2A-DB5C9ABE134D-44585'

In [85]:
bus_inf_s.keys().__len__()

22173

In [55]:
pcnTestDB.get_channel_inf(c_id='F8165186-C245-4465-8A2A-DB5C9ABE134D-44585')

{'OBJ_ID': ['F8165186-C245-4465-8A2A-DB5C9ABE134D-44585'],
 'NAME': ['E_2M[220kV越翎变:1框-2槽-调度专网（110kv奥封风电场-获嘉变）端口~时隙:1<>500kV获嘉变OptiX OSN 7500:1框-2槽-CPOS8/2/0端口~时隙:1-1-4-2]'],
 'CHANNEL_TYPE': ['1'],
 'RATE': ['5'],
 'A_RES_ID': ['2392F5E9-C562-4FEC-A860-D43216C74BF6-24762'],
 'Z_RES_ID': ['7BC33DA9-B603-41D6-A005-8C9651C56C4A-75842'],
 'A_STATION': ['085E98B2-66D4-4340-A21F-DC531C5F3A3D-76049'],
 'Z_STATION': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00152']}

In [62]:
'奥封风电场' in cha_inf_s['Z_STATION']

False

In [63]:
tmp_pd.loc[tmp_pd['OBJ_ID'] == 'F8165186-C245-4465-8A2A-DB5C9ABE134D-44585']

,OBJ_ID,NAME,CHANNEL_TYPE,RATE,A_RES_ID,Z_RES_ID,A_STATION,Z_STATION,A_STATION_NAME,Z_STATION_NAME
28539,F8165186-C245-4465-8A2A-DB5C9ABE134D-44585,E_2M[220kV越翎变:1框-2槽-调度专网（110kv奥封风电场-获嘉变）端口~时隙:...,1,5,2392F5E9-C562-4FEC-A860-D43216C74BF6-24762,7BC33DA9-B603-41D6-A005-8C9651C56C4A-75842,085E98B2-66D4-4340-A21F-DC531C5F3A3D-76049,F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00152,220kV越翎变,500kV获嘉变


In [64]:
pcnTestDB.get_business_inf(b_id='94FAF1DB-56F7-4C1E-B539-A542C1474E37-29055')

{'OBJ_ID': ['94FAF1DB-56F7-4C1E-B539-A542C1474E37-29055'],
 'FULL_NAME': ['豫/省公司自动化业务系统/(奥封风电场～500kV嘉和变) 调度自动化业务01'],
 'BUZ_TYPE': ['3'],
 'A_SITE_ID': ['4FF9B58D-A99F-46F8-82E2-9745D1B555D8-79715'],
 'Z_SITE_ID': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00255'],
 'DISPATCH_LEVEL': ['3'],
 'BUZ_RATE': ['2M']}

In [1]:
test_dict={
    '1':1,
    '2':2
}

['1','2'] in test_dict.keys()

TypeError: unhashable type: 'list'

In [18]:
pcnDBInf={
        'USER': 'PCN_TEST',
        'PASS': '784427618',
        'SID': '172.17.0.2:1521/LHR11G'
    } 
print(" PCN DataBase INF ".center(50, '*'))
for k in pcnDBInf.keys():
    if k != 'PASS':
        print("\t* {}: {}".format(k, pcnDBInf[k]))
print(" Successfully connected! ".center(50, '*'))


**************** PCN DataBase INF ****************
	* USER: PCN_TEST
	* SID: 172.17.0.2:1521/LHR11G
************ Successfully connected! *************
